In [ ]:
import requests
import pandas as pd

def fetch_candles(symbol: str, start_timestamp: int, end_timestamp: int, resolution: str = '1m'):
    """
    Fetches historical candle data from the Delta Exchange API.

    Args:
    - symbol (str): The symbol of the instrument (e.g., 'ETHUSD').
    - start_timestamp (int): The start timestamp in seconds.
    - end_timestamp (int): The end timestamp in seconds.
    - resolution (str): The resolution of the candles (default is '1m').

    Returns:
    - pd.DataFrame: A DataFrame containing the candle data.
    """

    # API Headers
    headers = {
        'Accept': 'application/json'
    }

    # Calculate chunk size based on 4000 candles and 1-minute resolution
    chunk_size = 4000 * 60  # in seconds

    # Initialize an empty list to store DataFrames
    dfs = []

    # Initialize current start time
    current_start = start_timestamp

    while current_start < end_timestamp:
        # Calculate end time for the current chunk
        current_end = min(current_start + chunk_size, end_timestamp)

        # Update parameters for the current chunk
        params = {
            'resolution': resolution,
            'symbol': symbol,
            'start': str(current_start),
            'end': str(current_end)
        }

        try:
            # Fetch data from API
            r = requests.get('https://api.india.delta.exchange/v2/history/candles', params=params, headers=headers)
            r.raise_for_status()

            # Convert response to JSON
            data = r.json()

            # Extract 'result' key from the dictionary if it exists
            if 'result' in data:
                print("API Response:")
                print(data['result'])  # Print the API response
                df = pd.DataFrame(data['result'])

                # Print the columns of the DataFrame
                print("DataFrame Columns:")
                print(df.columns)

                # Check if 'time' column exists
                if 'time' in df.columns:
                    # Convert 'time' to a readable datetime format
                    df['time'] = pd.to_datetime(df['time'], unit='s')

                    # Reorder columns if necessary
                    df = df[['time', 'open', 'high', 'low', 'close', 'volume']]

                    # Append to the list of DataFrames
                    dfs.append(df)
                else:
                    print(f"No 'time' column found in DataFrame for chunk {current_start} to {current_end}.")
            else:
                print(f"No 'result' key found in API response for chunk {current_start} to {current_end}.")
        except requests.RequestException as e:
            print(f"Error fetching data for chunk {current_start} to {current_end}: {e}")

        # Update current start time for the next chunk
        current_start = current_end + 1  # Adding 1 to avoid overlap

    # Concatenate all DataFrames
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        return None




In [ ]:
symbol = 'BTCUSD'

In [ ]:
df = fetch_candles(symbol, 1744755556, 1749469760)
df.to_csv(f'{symbol}.csv', index=False)